# Aufgabe 9.1

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 21 13:50:37 2021

@author: Chantal
"""
import os
import pandas as pd
import statsmodels.api as sm
import scipy.stats

In [2]:
# Auslesen der Daten
data = pd.read_excel("weeksworkedv2.xlsx","Sheet1")

# Abhängige Variablen und Regressoren
y = data[['weeks']]
x = data[['morekids', 'educ']]
x = pd.DataFrame(sm.add_constant(x.values, has_constant='add'), \
                    columns = ['Konstante'] + x.columns.tolist())

# Schätzung
weeksreg = sm.OLS(y,x).fit()

# Berechnung der Residuen des geschätzten Modells und Quadrierung der Residuen
resid=weeksreg.resid
residsq=resid*resid
#type(resid)
type(residsq)

pandas.core.series.Series

In [3]:
# Quadrierung der unabhängigen Variablen educ von Modell 1 (morekids als Dummy-Variable
# wird nicht quadriert)
data['educsq'] = data['educ']*data['educ']

# Definiere die Interaktion der beiden unabhängigen Variablen von Modell 1
data['educ_morekids'] = data['educ']*data['morekids']

# Festsetzen der unabhängigen Variablen der White-Hilfsregression. Die Hilfsregression
# enthält immer die beiden Regressoren des ursprünglichen Modells, die beiden quadrierten
# Regressoren des ursprünglichen Modells und die Interaktion der beiden Regressoren
XVarWhite = data[['educ','morekids','educsq','educ_morekids']]
XVarWhite = pd.DataFrame(sm.add_constant(XVarWhite.values, has_constant='add'),\
             columns = ['Konstante'] + XVarWhite.columns.tolist())

# Festsetzen der abhängigen Variable der Hilfsregression, hier also die quadrierten
# Residuen von weeksreg
YVarWhite=residsq

# Schätzung der White-Hilfsregession
weeks_White = sm.OLS(YVarWhite,XVarWhite).fit()

## b)


In [4]:
# Berechnung des realisierten Wertes des White-Testes anhand der Anzahl der Beobachtungen
# der Hilfsregression und des Bestimmtheitsmaßes der Hilfsregression
R2_White=weeks_White.rsquared
#k_White= weeks_White.df_model+weeks_White.k_constant
#n_White=int(weeks_White.df_resid +k_White )
White_act=weeks_White.nobs*R2_White
#print("R2: ",R2_White)
#print("n_white: ", n_White)
print('Realisierter Wert zum White-Test', White_act)

Realisierter Wert zum White-Test 3.4980967004657


In [5]:
print(weeks_White.df_model)
print(weeks_White.k_constant)
print(weeks_White.nobs)
print(weeks_White.df_resid)

4.0
1
273.0
268.0



## c)


In [7]:
# Berechnung des kritischen Wertes anhand der Chi-Verteilung zum Signifikanzniveau 1%
#und der Anzahl der Freiheitsgrade k_White-1, wobei k_White für die Anzahl der geschätzten
#Koeffizienten der Hilfsregression steht.
alphaWhite=0.01
chi_krit=scipy.stats.chi2.ppf((1-alphaWhite),weeks_White.df_model)
print('Kritischer Wert zum White-Test', chi_krit)

Kritischer Wert zum White-Test 13.276704135987622


# Aufgabe 9.3

In [9]:
# Auslesen der Daten
data = pd.read_excel("trmgpa.xlsx","Sheet1")

# Abhängige Variablen und Regressoren
y = data[['trmgpa']]
x = data[['female', 'frstsem']]
x = pd.DataFrame(sm.add_constant(x.values, has_constant='add'), \
                    columns = ['const'] + x.columns.tolist())

# Schätzung
trmgpa = sm.OLS(y,x).fit()
print(trmgpa.summary())
print('Geschätzte Koeffizienten zu trmgpa \n', trmgpa.params)
print('Standardfehler zu trmgpa \n', trmgpa.bse)
print('Robuste Standardfehler zu trmgpa \n', trmgpa.HC0_se)


                            OLS Regression Results                            
Dep. Variable:                 trmgpa   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                     43.89
Date:                Sun, 13 Mar 2022   Prob (F-statistic):           1.00e-18
Time:                        18:23:39   Log-Likelihood:                -793.98
No. Observations:                 732   AIC:                             1594.
Df Residuals:                     729   BIC:                             1608.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2110      0.032     69.054      0.0